In [3]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@5a6c4b73
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSe

In [5]:
val df_1 = spark.read.option("header", true).option("inferSchema",true).csv("AB_data_1.csv")
val df_2 = spark.read.option("header", true).option("inferSchema",true).csv("editor_open.csv")
val df_3 = spark.read.option("header", true).option("inferSchema",true).csv("editor_done.csv")

df_1: DataFrame = [variant: string, device_id: string ... 1 more field]
df_2: DataFrame = [device_id: string, event_type: string ... 5 more fields]
df_3: DataFrame = [device_id: string, event_type: string ... 4 more fields]

In [6]:
val df2 = df_2.drop("country_code", "platform", "source").withColumnRenamed("event_type", "event_eo").
      withColumnRenamed("timestamp", "timestamp_eo")
val df3 = df_3.drop("country_code", "platform").withColumnRenamed("event_type", "event_ed").
      withColumnRenamed("timestamp", "timestamp_ed")


df2: DataFrame = [device_id: string, event_eo: string ... 2 more fields]
df3: DataFrame = [device_id: string, event_ed: string ... 2 more fields]

In [8]:
val df_new = df2.join(df3, df2("device_id")<=>df3("device_id") && df2("editor_sid")<=>
             df3("editor_sid") && df2("timestamp_eo") < df3("timestamp_ed"), "left")
val df_fin = df_new.drop(df3("device_id")).drop( df3("editor_sid"))

df_new: DataFrame = [device_id: string, event_eo: string ... 6 more fields]
df_fin: DataFrame = [device_id: string, event_eo: string ... 4 more fields]

In [9]:
df_fin.show(5)

+--------------------+-----------+-------------------+--------------------+--------+------------+
|           device_id|   event_eo|       timestamp_eo|          editor_sid|event_ed|timestamp_ed|
+--------------------+-----------+-------------------+--------------------+--------+------------+
|de2522a5-06ad-441...|editor_open|2019-12-19 05:56:33|7bbf5f5d-5eca-4b0...|    null|        null|
|1c5d7b92-f5a4-430...|editor_open|2019-12-19 05:46:32|5d092b06-7cf7-409...|    null|        null|
|F6232B9D-6E26-404...|editor_open|2019-12-19 06:37:31|f6232b9d-6e26-404...|    null|        null|
|67a8d922-3b7a-4ca...|editor_open|2019-12-19 08:55:08|063eb382-1722-479...|    null|        null|
|d5a2beb0-f196-4b0...|editor_open|2019-12-19 08:50:42|c655674c-9fed-47c...|    null|        null|
+--------------------+-----------+-------------------+--------------------+--------+------------+
only showing top 5 rows



In [45]:
val df_final = df_1.join(df_fin, df_1("device_id")<=>df_fin("device_id") && df_1("timestamp")<
           df_fin("timestamp_eo"), "left").withColumnRenamed("editor_sid", "session_id")
val frame = df_final.drop(df_fin("device_id")).drop(df_fin("timestamp_eo")).
        drop(df_fin("timestamp_ed"))
frame.agg(min(col("timestamp")), max(col("timestamp"))).show()

+-------------------+-------------------+
|     min(timestamp)|     max(timestamp)|
+-------------------+-------------------+
|2019-12-20 12:06:50|2019-12-23 02:47:37|
+-------------------+-------------------+



df_final: DataFrame = [variant: string, device_id: string ... 7 more fields]
frame: DataFrame = [variant: string, device_id: string ... 4 more fields]

In [54]:
frame.show(10)
val summary = frame.groupBy("variant").agg(countDistinct(col("device_id")), 
              countDistinct(col("session_id")), count(col("event_eo")), count(col("event_ed")))

+----------+--------------------+-------------------+-----------+--------------------+-----------------+
|   variant|           device_id|          timestamp|   event_eo|          session_id|         event_ed|
+----------+--------------------+-------------------+-----------+--------------------+-----------------+
|with_remix|1e090ca0-8d8e-451...|2019-12-22 21:26:05|editor_open|ccd62571-f4ce-4a4...|             null|
|with_remix|1e090ca0-8d8e-451...|2019-12-22 21:26:05|editor_open|ccd62571-f4ce-4a4...|editor_done_click|
|with_remix|1e090ca0-8d8e-451...|2019-12-22 21:26:05|editor_open|ccd62571-f4ce-4a4...|             null|
|with_remix|42f5b1ee-1b9c-42f...|2019-12-21 19:51:24|       null|                null|             null|
|  original|7290d6ca-6abc-459...|2019-12-21 06:03:36|editor_open|26976319-e5b5-46e...|editor_done_click|
|  original|7290d6ca-6abc-459...|2019-12-21 06:03:36|editor_open|26976319-e5b5-46e...|             null|
|  original|7290d6ca-6abc-459...|2019-12-21 06:03:36|ed

summary: DataFrame = [variant: string, count(DISTINCT device_id): bigint ... 3 more fields]

In [55]:
summary.show()

+----------+-------------------------+--------------------------+---------------+---------------+
|   variant|count(DISTINCT device_id)|count(DISTINCT session_id)|count(event_eo)|count(event_ed)|
+----------+-------------------------+--------------------------+---------------+---------------+
|with_remix|                      914|                      2670|           2890|           1187|
|  original|                      917|                      2862|           3051|           1339|
+----------+-------------------------+--------------------------+---------------+---------------+



In [56]:
//counts per each event and conversion rate
val summary1 = summary.withColumn("conv_eo_ed", col("count(event_ed)")*100/col("count(event_eo)"))
summary1.show()

+----------+-------------------------+--------------------------+---------------+---------------+------------------+
|   variant|count(DISTINCT device_id)|count(DISTINCT session_id)|count(event_eo)|count(event_ed)|        conv_eo_ed|
+----------+-------------------------+--------------------------+---------------+---------------+------------------+
|with_remix|                      914|                      2670|           2890|           1187| 41.07266435986159|
|  original|                      917|                      2862|           3051|           1339|43.887250081940344|
+----------+-------------------------+--------------------------+---------------+---------------+------------------+



summary1: DataFrame = [variant: string, count(DISTINCT device_id): bigint ... 4 more fields]

In [47]:
//ret 1 day!!!
val ret_original = retention(frame.filter(col("variant")==="original"), 1)
val ret_remix = retention(frame.filter(col("variant")==="with_remix"), 1)
//????

+-----------------------------------+
|avg_retention_rate_over_time_period|
+-----------------------------------+
|                               -0.6|
+-----------------------------------+

()
+-----------------------------------+
|avg_retention_rate_over_time_period|
+-----------------------------------+
|                               -0.6|
+-----------------------------------+

()


In [59]:
//mean, stdev summary (editor_open, editor_done)
frame.filter(col("variant")==="original").groupBy("device_id").agg(count(col("event_eo"))).
        describe().show()
frame.filter(col("variant")==="with_remix").groupBy("device_id").agg(count(col("event_eo"))).
        describe().show()
frame.filter(col("variant")==="original").groupBy("device_id").agg(count(col("event_ed"))).
        describe().show()
frame.filter(col("variant")==="with_remix").groupBy("device_id").agg(count(col("event_ed"))).
        describe().show()

+-------+--------------------+-----------------+
|summary|           device_id|  count(event_eo)|
+-------+--------------------+-----------------+
|  count|                 917|              917|
|   mean|                null|3.327153762268266|
| stddev|                null|7.707100868008031|
|    min|0032fa0b-7143-41d...|                0|
|    max|fff6e772-5651-49d...|              142|
+-------+--------------------+-----------------+

+-------+--------------------+------------------+
|summary|           device_id|   count(event_eo)|
+-------+--------------------+------------------+
|  count|                 914|               914|
|   mean|                null|3.1619256017505473|
| stddev|                null| 7.237485706708468|
|    min|00589dc6-8909-468...|                 0|
|    max|fffcdd0e-61fd-4fe...|                95|
+-------+--------------------+------------------+

+-------+--------------------+------------------+
|summary|           device_id|   count(event_ed)|
+------

In [58]:
//total counts
group_count(df_1, Array("variant"), Array("device_id")).show()

+----------+---------------+
|   variant|count_device_id|
+----------+---------------+
|with_remix|            914|
|  original|            917|
+----------+---------------+



In [14]:
def group_count(dataframe:DataFrame, group_col:Array[String], count:Array[String]):DataFrame = {
        var count_col:Array[Column] = count.map(c => countDistinct(col(c)).as("count_"+c))
        var grouped_frame = dataframe.groupBy(group_col.head, group_col.tail:_*).agg(count_col.head, count_col.tail:_*)
        grouped_frame
}
def retention(dataframe:DataFrame, number_of_days:Int):Unit = {
        var frame = dataframe.withColumn("date", to_date(dataframe("timestamp")))
        var device_id_col = frame("device_id")
        var date_col = frame("date")
        var df_1 = group_count(frame, Array("date"), Array("device_id")).withColumnRenamed("date", "date1")
        var df_2 = frame.select("date", "device_id").groupBy(date_col).agg(collect_set(device_id_col).as("device_id_set"))
        var df_join = df_1.join(df_2, df_1("date1") <=> df_2("date"), "inner")
        var window_spec = Window.orderBy(date_col)
        var df_3 = df_join.withColumn("set_after_days", lead("device_id_set", number_of_days) over window_spec)
        var df_4 = df_3.withColumn("common_id_set", array_intersect(col("device_id_set"), col("set_after_days")))
        var df_5 = df_4.withColumn("count_ret_ids", size(col("common_id_set"))).withColumn("retention_rate",
                    round(col("count_ret_ids")*100/col("count_device_id"), 2)).drop("date1", "device_id_set", 
                    "set_after_days", "common_id_set")
        var avg_ret_rate = df_5.agg(round(avg(col("retention_rate")), 1).as("avg_retention_rate_over_time_period"))
        println(avg_ret_rate.show())
    }

defined function group_count
defined function retention